<a href="https://colab.research.google.com/github/darkfapper/Pandoras_Box/blob/main/TESTE_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0
y_train_one_hot = tf.keras.utils.to_categorical(y_train)
y_test_one_hot = tf.keras.utils.to_categorical(y_test)

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp

# Usage example
input_size = 784
fc2_size = 256
fc3_size = 128
fc4_size = 64
output_size = 10

neg_log_likelihood = lambda y, p_y: -p_y.log_prob(y)
tfd = tfp.distributions
tfpl = tfp.layers

## Testar usando 1
NUM_TRAIN_EXAMPLES =  x_train.shape[0] #1
kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) / tf.cast(NUM_TRAIN_EXAMPLES, dtype=tf.float32))

class BayesianNetwork(tf.keras.Model):
    def __init__(self, input_size, fc2_size, fc3_size, fc4_size, output_size):
        super(BayesianNetwork, self).__init__()
        self.fc1 = tfp.layers.DenseFlipout(input_size, activation='relu',kernel_divergence_fn=kl_divergence_function,bias_divergence_fn=kl_divergence_function)
        self.fc2 = tfp.layers.DenseFlipout(fc2_size, activation='relu',kernel_divergence_fn=kl_divergence_function,bias_divergence_fn=kl_divergence_function)
        self.fc3 = tfp.layers.DenseFlipout(fc3_size, activation='relu',kernel_divergence_fn=kl_divergence_function,bias_divergence_fn=kl_divergence_function)
        self.fc4 = tfp.layers.DenseFlipout(fc4_size, activation='relu',kernel_divergence_fn=kl_divergence_function,bias_divergence_fn=kl_divergence_function)
        self.fc5 = tfp.layers.DenseFlipout(tfp.layers.OneHotCategorical.params_size(output_size),activation=None,kernel_divergence_fn=kl_divergence_function,bias_divergence_fn=kl_divergence_function)
        self.fc6 = tfp.layers.OneHotCategorical(output_size)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.fc5(x)
        x = self.fc6(x)
        return x





model = BayesianNetwork(input_size, fc2_size, fc3_size, fc4_size, output_size)


# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=neg_log_likelihood,
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_one_hot, epochs=50, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test_one_hot)
print('Test accuracy:', test_acc)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/layers/util.py:98: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/layers/util.py:108: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


1500/1500 [==============================] - ETA: 0s - loss: 31.8177 - accuracy: 0.8342